In [16]:
%reset -f
import os
import sys
import importlib
import logging
from pathlib import Path
import polars as pl
import torch
import importlib
import traceback
import inspect
import random
from dataclasses import asdict
from typing import Dict, Optional
import lightgbm as lgb
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Setup project path
proj = Path.cwd()
if (proj / "src").exists():
    root = proj
elif (proj.parent / "src").exists():
    root = proj.parent
else:
    root = next(p for p in [proj, *proj.parents] if (p / "src").exists())

# Set up python path
os.chdir(root)
if str(root) not in sys.path:
    sys.path.insert(0, str(root))
logger.info(f"Project root configured: {root}")

# Verify critical paths exist
for path in ["src", "data", "data/raw", "data/processed", "notebooks"]:
    if not (root / path).exists():
        raise RuntimeError(f"Missing required path: {root / path}")

2025-11-26 09:11:23,485 - INFO - Project root configured: e:\OneDrive\Documents\Courses\Artificial Intelligence\Project\UF_CAP4261_F25_TEAM9


In [ ]:
!pip install lightgbm

In [17]:
from src.models.player_dataset import load_processed_data, join_teamframe, build_player_id_map
from src.models.train_player_model import (
    FEATURE_COLS,
    RANDOM_SEED,
    set_seed,
    agentic_search_frames,
    train_final_frame_model,
    predict_frames_on_test,
)

set_seed(RANDOM_SEED)


In [18]:
players_train, players_test, teamframe_train, teamframe_test = load_processed_data()

players_train_joined = join_teamframe(players_train, teamframe_train)
players_test_joined  = join_teamframe(players_test, teamframe_test)

player_id_map = build_player_id_map(players_train_joined)

print("players_train_joined:", players_train_joined.shape)
print("players_test_joined:", players_test_joined.shape)
print("num unique players:", len(player_id_map))


players_train_joined: (4880579, 58)
players_test_joined: (49753, 58)
num unique players: 1384


In [7]:
players_train_joined.head()

game_id,play_id,frame_id,nfl_id,player_side,player_role,player_position,player_to_predict,x_norm,y_norm,dir_norm,o_norm,s,a,vx,vy,ax,ay,dir_sin,dir_cos,o_sin,o_cos,height_in,player_weight,player_age,player_bmi,absolute_yardline_number,num_frames_output,ball_land_x,ball_land_y,speed_rolling_mean_3,speed_rolling_std_3,accel_rolling_mean_3,accel_rolling_std_3,speed_rolling_mean_5,speed_rolling_std_5,accel_rolling_mean_5,accel_rolling_std_5,angular_velocity,delta_x,delta_y,cumulative_distance,formation_width,formation_depth,formation_x_mean,formation_y_mean,team_spread_mean,depth_std,width_std,nn_teammate_mean,nn_opponent_mean,coverage_density_mean,opp_within_3yds_sum,opp_within_5yds_sum,opp_within_7yds_sum,tm_within_3yds_sum,tm_within_5yds_sum,tm_within_7yds_sum
i64,i64,i32,i64,str,str,str,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023090700,1001,1,41325,"""Offense""","""Other Route Runner""","""RB""",false,104.38,26.83,264.32,350.33,0.14,0.38,-0.013856,-0.139313,-0.03761,-0.378134,-0.99509,-0.098972,-0.167973,0.985792,69.0,205,NaN,30.269901,109,12,115.610001,34.799999,null,null,null,null,null,null,null,null,null,null,null,null,16.58,4.17,106.743333,24.618333,5.570082,1.775992,6.382427,3.088956,4.216166,0.870827,6.0,7.0,12.0,2.0,8.0,14.0
2023090700,1001,2,41325,"""Offense""","""Other Route Runner""","""RB""",false,104.37,26.84,321.47,3.49,0.43,1.36,0.336381,-0.267857,1.063904,-0.847177,-0.622924,0.782282,0.060874,0.998145,69.0,205,NaN,30.269901,109,12,115.610001,34.799999,null,null,null,null,null,null,null,null,57.15,-0.01,0.01,0.014142,16.56,4.21,106.756667,24.611667,5.594439,1.802739,6.393833,3.061613,4.200996,0.873751,6.0,7.0,12.0,2.0,8.0,14.0
2023090700,1001,3,41325,"""Offense""","""Other Route Runner""","""RB""",false,104.34,26.88,333.38,13.55,0.78,2.34,0.697318,-0.349496,2.091955,-1.048487,-0.448071,0.893998,0.234294,0.972166,69.0,205,NaN,30.269901,109,12,115.610001,34.799999,0.45,0.320468,1.36,0.98,null,null,null,null,11.91,-0.03,0.04,0.064142,16.52,4.3,106.785,24.613333,5.634313,1.850889,6.410653,3.026665,4.162334,0.880733,6.0,7.0,12.0,2.0,8.0,12.0
2023090700,1001,4,41325,"""Offense""","""Other Route Runner""","""RB""",false,104.3,26.95,340.18,21.91,1.14,3.69,1.072469,-0.386536,3.471414,-1.251155,-0.339066,0.940762,0.37315,0.927771,69.0,205,NaN,30.269901,109,12,115.610001,34.799999,0.783333,0.355012,2.463333,1.169886,null,null,null,null,6.8,-0.04,0.07,0.144765,16.48,4.43,106.811667,24.618333,5.687051,1.915635,6.433125,3.000016,4.12731,0.888186,6.0,7.0,12.0,2.0,8.0,10.0
2023090700,1001,5,41325,"""Offense""","""Other Route Runner""","""RB""",false,104.26,27.08,344.03,32.8,1.61,3.88,1.547863,-0.442966,3.730255,-1.06752,-0.275134,0.961406,0.541708,0.840567,69.0,205,NaN,30.269901,109,12,115.610001,34.799999,1.176667,0.416213,3.303333,0.839663,0.82,0.579353,2.33,1.499633,3.85,-0.04,0.13,0.280779,16.45,4.58,106.841667,24.628333,5.762022,1.997423,6.470621,2.997089,4.068832,0.89826,6.0,7.0,12.0,2.0,8.0,12.0


In [24]:
players_train_joined.select([
    *[pl.col(c).is_null().sum().alias(f"{c}_nulls") for c in FEATURE_COLS]
]).to_pandas().T.head(40)


,0
x_norm_nulls,0
y_norm_nulls,0
dir_norm_nulls,0
o_norm_nulls,0
s_nulls,0
a_nulls,0
vx_nulls,0
vy_nulls,0
ax_nulls,0
ay_nulls,0


In [19]:
# Agentic search (frame-based)
best_agentic_frames = agentic_search_frames(players_train_joined, player_id_map, FEATURE_COLS, n_sequences=3000, trials=100)
hist = best_agentic_frames["history"]
hist_df = pd.DataFrame(hist)

display(hist_df)

plt.figure(figsize=(6, 4))
plt.plot(hist_df["trial"], hist_df["cv_rmse"], marker="o")
plt.xlabel("Trial")
plt.ylabel("CV RMSE")
plt.title("Frame-level Agentic Search: CV RMSE by Trial")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
plt.scatter(hist_df["model_dim"], hist_df["cv_rmse"])
plt.xlabel("model_dim")
plt.ylabel("CV RMSE")
plt.title("CV RMSE vs model_dim")
plt.grid(True)
plt.tight_layout()
plt.show()

# Train final frame model on a slightly larger subset
frame_model = train_final_frame_model(players_train_joined, player_id_map, best_agentic_frames["cfg"], n_sequences=600)

# Save and predict on test
import os, torch
os.makedirs("models", exist_ok=True)
torch.save(
    {"state_dict": frame_model.state_dict(),
     "cfg": asdict(best_agentic_frames["cfg"]),
     "player_id_map": player_id_map},
    "models/player_physics_transformer_frames_agentic.pt",
)

predict_frames_on_test(
    frame_model,
    players_test_joined,
    player_id_map,
    out_path="models/frame_level_predictions_agentic.csv",
)


Frame dataset size (sequences): 791

[Frames Agentic] Trial 1/100 with cfg: TransformerFramesConfig(input_dim=31, model_dim=128, num_layers=1, num_heads=2, num_players=1384, dropout=0.1)


e:\OneDrive\Documents\Courses\Artificial Intelligence\Project\UF_CAP4261_F25_TEAM9\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  CV RMSE (per-frame): 48.2409
  👉 New best frame config: {'cfg': TransformerFramesConfig(input_dim=31, model_dim=128, num_layers=1, num_heads=2, num_players=1384, dropout=0.1), 'score': 48.240891774495445}

[Frames Agentic] Trial 2/100 with cfg: TransformerFramesConfig(input_dim=31, model_dim=96, num_layers=1, num_heads=2, num_players=1384, dropout=0.1)
  CV RMSE (per-frame): 49.0353

[Frames Agentic] Trial 3/100 with cfg: TransformerFramesConfig(input_dim=31, model_dim=160, num_layers=1, num_heads=4, num_players=1384, dropout=0.1)
  CV RMSE (per-frame): 47.3765
  👉 New best frame config: {'cfg': TransformerFramesConfig(input_dim=31, model_dim=160, num_layers=1, num_heads=4, num_players=1384, dropout=0.1), 'score': 47.376522064208984}

[Frames Agentic] Trial 4/100 with cfg: TransformerFramesConfig(input_dim=31, model_dim=192, num_layers=2, num_heads=2, num_players=1384, dropout=0.1)
  CV RMSE (per-frame): 45.6906
  👉 New best frame config: {'cfg': TransformerFramesConfig(input_dim=31,

AssertionError: embed_dim must be divisible by num_heads